In [1]:
import time
import json
import os
import pandas as pd
import numpy as np
import flwr as fl
import pickle


from sklearn.model_selection import train_test_split

from hydra import initialize, compose
from omegaconf import OmegaConf, DictConfig

from logging import INFO, DEBUG
from flwr.common.logger import log


from src.models.evaluation_metrics import custom_acc_mc, custom_acc_binary
from src.data.dataset_info import datasets

with initialize(version_base=None, config_path="conf/"):
    cfg = compose(config_name='config.yaml')
    print(OmegaConf.to_yaml(cfg))


dataset = datasets[0]

folder_path = "./datasets/gdlc/"
# folder_path = "./datasets/dbp/"

lr_decay = True
early_stopping = False

pca = True
digraph_centralities = False
Multi_graph_centralities = False

learning_rate = 0.0001
LAMBD_1 = 0.0001
LAMBD_2 = 0.001

multi_class: false
with_network_features: false
n_clients: 5
n_rounds: 20
config_fit:
  momentum: 0.9
  local_epochs: 5
  batch_size: 256



In [2]:
type(OmegaConf.to_container(cfg))

dict

In [3]:
OmegaConf.to_container(cfg)

{'multi_class': False,
 'with_network_features': False,
 'n_clients': 5,
 'n_rounds': 20,
 'config_fit': {'momentum': 0.9, 'local_epochs': 5, 'batch_size': 256}}

In [4]:
dtime = time.strftime("%Y%m%d-%H%M%S")
dtime

'20240826-195245'

In [5]:
clients_paths = [
    folder_path + "client_0.parquet",
    folder_path + "client_1.parquet",
    folder_path + "client_2.parquet",
    folder_path + "client_3.parquet",
    folder_path + "client_4.parquet",
    folder_path + "client_5.parquet",
    folder_path + "client_6.parquet",
    folder_path + "client_7.parquet",
    folder_path + "test.parquet"
]

# clients_paths = [
#     folder_path + "client_0_pca.parquet",
#     folder_path + "client_1_pca.parquet",
#     folder_path + "client_2_pca.parquet",
#     folder_path + "client_3_pca.parquet",
#     folder_path + "client_4_pca.parquet",
#     folder_path + "client_5_pca.parquet",
#     folder_path + "client_6_pca.parquet",
#     folder_path + "client_7_pca.parquet",
#     folder_path + "test.parquet"
# ]

# clients_paths = [
#     folder_path + "client_0_global_pca.parquet",
#     folder_path + "client_1_global_pca.parquet",
#     folder_path + "client_2_global_pca.parquet",
#     folder_path + "client_3_global_pca.parquet",
#     folder_path + "client_4_global_pca.parquet",
#     folder_path + "client_5_global_pca.parquet",
#     folder_path + "client_6_global_pca.parquet",
#     folder_path + "client_7_global_pca.parquet",
#     folder_path + "test_global_pca.parquet"
# ]

# Data Loading and Preprocessing

In [6]:
with open(folder_path + "added_columns.pkl", 'rb') as f:
    centralities_columns, pca_columns = pickle.load(f)
    
centralities_columns, pca_columns

([['src_betweenness',
   'dst_betweenness',
   'src_local_betweenness',
   'dst_local_betweenness',
   'src_degree',
   'dst_degree',
   'src_local_degree',
   'dst_local_degree',
   'src_eigenvector',
   'dst_eigenvector',
   'src_closeness',
   'dst_closeness',
   'src_pagerank',
   'dst_pagerank',
   'src_local_pagerank',
   'dst_local_pagerank',
   'src_k_core',
   'dst_k_core',
   'src_k_truss',
   'dst_k_truss',
   'src_Comm',
   'dst_Comm'],
  ['src_betweenness',
   'dst_betweenness',
   'src_local_betweenness',
   'dst_local_betweenness',
   'src_pagerank',
   'dst_pagerank',
   'src_local_pagerank',
   'dst_local_pagerank',
   'src_k_core',
   'dst_k_core',
   'src_k_truss',
   'dst_k_truss',
   'src_Comm',
   'dst_Comm'],
  ['src_betweenness',
   'dst_betweenness',
   'src_local_betweenness',
   'dst_local_betweenness',
   'src_pagerank',
   'dst_pagerank',
   'src_local_pagerank',
   'dst_local_pagerank',
   'src_k_core',
   'dst_k_core',
   'src_k_truss',
   'dst_k_truss',


In [7]:
test = pd.read_parquet(folder_path + "test.parquet")
print(test.columns)

Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       ...
       'src_global_pagerank', 'dst_global_pagerank', 'src_k_core',
       'dst_k_core', 'src_k_truss', 'dst_k_truss', 'src_mv', 'dst_mv',
       'global_pca_1', 'global_pca_2'],
      dtype='object', length=109)


In [8]:
client_data = []
for client_path in clients_paths:
    client_data.append(pd.read_parquet(client_path))

In [9]:
client_columns = []

for client_df in client_data:
    client_columns.append(set(client_df.columns))

common_columns = set.intersection(*client_columns)

differences = [columns - common_columns for columns in client_columns]

# Display the columns of each client, the intersection, and the differences
#for idx, columns in enumerate(client_columns):
 #   print(f"Client {idx} columns: {columns}")

print(f"\nIntersection of columns across all clients: {common_columns}")

for idx, diff in enumerate(differences):
    print(f"Difference in columns for client {idx}: {diff}")


Intersection of columns across all clients: {'RST Flag Cnt', 'Pkt Len Var', 'Subflow Fwd Byts', 'Class', 'URG Flag Cnt', 'ECE Flag Cnt', 'Bwd IAT Max', 'Dst Port', 'Fwd IAT Mean', 'Bwd Blk Rate Avg', 'global_pca_1', 'Tot Fwd Pkts', 'Fwd Seg Size Avg', 'Protocol', 'src_betweenness', 'src_k_core', 'Flow IAT Max', 'Idle Min', 'Pkt Size Avg', 'Fwd Pkts/s', 'Dst IP', 'Bwd Pkt Len Std', 'Flow IAT Std', 'Flow IAT Min', 'Bwd Pkt Len Min', 'Fwd URG Flags', 'Subflow Fwd Pkts', 'dst_pagerank', 'Bwd URG Flags', 'Idle Max', 'TotLen Fwd Pkts', 'Flow Byts/s', 'Pkt Len Mean', 'Bwd Pkt Len Max', 'Idle Std', 'Fwd Header Len', 'Fwd PSH Flags', 'Fwd Act Data Pkts', 'FIN Flag Cnt', 'Fwd IAT Min', 'Fwd Pkts/b Avg', 'ACK Flag Cnt', 'Label', 'Fwd IAT Tot', 'Init Bwd Win Byts', 'Active Std', 'Idle Mean', 'dst_betweenness', 'Pkt Len Min', 'Fwd Pkt Len Std', 'Bwd IAT Min', 'CWE Flag Count', 'Bwd IAT Tot', 'Fwd Blk Rate Avg', 'Down/Up Ratio', 'Bwd PSH Flags', 'Bwd IAT Mean', 'Timestamp', 'Flow Duration', 'Active

In [10]:
# the input dimension of the training set
# input_dim = df.shape[1] - len(drop_columns) - len(weak_columns) - 1  # for the label_column
  
# specifying the number of classes, since it is different from one dataset to another and also if binary or multi-class classification
classes_set = {"benign", "attack"}
labels_names = {0: "benign", 1: "attack"}
num_classes = 2
if cfg.multi_class:
    with open(folder_path + "labels_names.pkl", 'rb') as f:
        labels_names, classes_set = pickle.load(f)
    num_classes = len(classes_set)
    
labels_names = {int(k): v for k, v in labels_names.items()}

print(f"==>> classes_set: {classes_set}")
print(f"==>> num_classes: {num_classes}")
print(f"==>> labels_names: {labels_names}")

==>> classes_set: {'attack', 'benign'}
==>> num_classes: 2
==>> labels_names: {0: 'benign', 1: 'attack'}


In [11]:


if cfg.multi_class:
    test[dataset.label_col] = test[dataset.class_num_col]
    
    
test.drop(centralities_columns[-1], axis=1, inplace=True)
test.drop(pca_columns, axis=1, inplace=True)
# test.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
# test.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
# test.drop(["pca_1", "pca_2"], axis=1, inplace=True)
if not cfg.multi_class:
    test_by_class = {}
    classes = test[dataset.class_col].unique()
    for class_value in classes:
        test_class = test[test[dataset.class_col] == class_value].copy()
        test_class.drop(dataset.drop_columns, axis=1, inplace=True)
        test_class.drop(dataset.weak_columns, axis=1, inplace=True)
        test_class.reset_index(drop=True, inplace=True)

        test_class_labels = test_class[dataset.label_col].to_numpy()
        test_class = test_class.drop([dataset.label_col], axis=1).to_numpy()

        test_by_class[class_value] = (test_class, test_class_labels)
    
    
test.drop(dataset.drop_columns, axis=1, inplace=True)
test.drop(dataset.weak_columns, axis=1, inplace=True)
test.reset_index(drop=True, inplace=True)
    
test_labels = test[dataset.label_col].to_numpy()
test = test.drop([dataset.label_col], axis=1).to_numpy()
input_dim = test.shape[1]
    
for i in range(len(client_data)):
    
    cdata = client_data[i]
    
    if cfg.multi_class:
        cdata[dataset.label_col] = cdata[dataset.class_num_col]
        
    cdata.drop(centralities_columns[i], axis=1, inplace=True)
    cdata.drop(pca_columns, axis=1, inplace=True)
    # cdata.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
    # cdata.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
    # cdata.drop(["pca_1", "pca_2"], axis=1, inplace=True)

    cdata.drop(dataset.drop_columns, axis=1, inplace=True)
    cdata.drop(dataset.weak_columns, axis=1, inplace=True)
    cdata.reset_index(drop=True, inplace=True)
    
    c_train, c_test = train_test_split(cdata, test_size=0.1)

    y_train = c_train[dataset.label_col].to_numpy()
    x_train = c_train.drop([dataset.label_col], axis=1).to_numpy()
    y_test = c_test[dataset.label_col].to_numpy()
    x_test = c_test.drop([dataset.label_col], axis=1).to_numpy()
    
    client_data[i] = (x_train, y_train, x_test, y_test)

# Model

In [12]:
from keras import layers, models, Input, regularizers, callbacks, metrics, optimizers, initializers
# from src.models.evaluation_metrics import f1_m

def create_keras_model(input_shape, alpha = learning_rate):
    model = models.Sequential()
    
    model.add(layers.Conv1D(80, kernel_size=3,
                activation="relu", input_shape=(input_shape, 1), kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.MaxPooling1D())
    model.add(layers.LayerNormalization(axis=1))
    # .L1L2(l1=LAMBD_1, l2=LAMBD_2)
    model.add(layers.Conv1D(80, 3, activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.MaxPooling1D())
    model.add(layers.LayerNormalization(axis=1))
    
    # model.add(layers.LSTM(units=80,
    #                         activation='relu',
    #                         kernel_regularizer=regularizers.L1L2(l1=LAMBD_1, l2=LAMBD_2),
    #                         recurrent_regularizer=regularizers.L1L2(l1=LAMBD_1, l2=LAMBD_2),
    #                         bias_regularizer=regularizers.L1L2(l1=LAMBD_1, l2=LAMBD_2),
    #                         return_sequences=False,
    #                         ))
    # model.add(layers.LayerNormalization(axis=1))
    
    model.add(layers.Flatten())

    model.add(layers.Dense(200,activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.LayerNormalization(axis=1))
    model.add(layers.Dense(200,activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.LayerNormalization(axis=1))
    model.add(layers.Dense(80,activation='relu', kernel_regularizer=regularizers.L2(l2=LAMBD_2)))
    model.add(layers.LayerNormalization(axis=1))

    if cfg.multi_class:
        model.add(layers.Dense(num_classes, activation='softmax'))
        model.compile(optimizer=optimizers.Adam(learning_rate=alpha),
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])
    else:
        model.add(layers.Dense(1, activation='sigmoid'))
        model.compile(optimizer=optimizers.Adam(learning_rate=alpha),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])
    
    
    return model


In [13]:
model = create_keras_model(input_dim)
model.summary()

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 36, 80)         │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 18, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, 18, 80)         │            36 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 16, 80)         │        19,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 8, 80)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ (None, 8, 80)          │            16 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │       128,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_2           │ (None, 200)            │           400 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 200)            │        40,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_3           │ (None, 200)            │           400 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 80)             │        16,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_4           │ (None, 80)             │           160 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 205,173 (801.46 KB)

 Trainable params: 205,173 (801.46 KB)

 Non-trainable params: 0 (0.00 B)

# FL

## FL settings

In [14]:
results_final = {}

results_final["model"] = {}
results_final["configuration"] = {
    "folder_path": folder_path,
    "lr_decay": lr_decay,
    "early_stopping": early_stopping,
    "pca": pca,
    "digraph_centralities": digraph_centralities,
    "Multi_graph_centralities": Multi_graph_centralities,
    "learning_rate": learning_rate,
    "LAMBD_1": LAMBD_1,
    "LAMBD_2": LAMBD_2,
    "cfg": OmegaConf.to_container(cfg)
}

results_final["baseline"] = {}
results_final["baseline"]["accuracy"] = {}
results_final["baseline"]["f1s"] = {}

results_final["centralities - PCA"] = {}
results_final["centralities - PCA"]["accuracy"] = {}
results_final["centralities - PCA"]["f1s"] = {}

results_final["centralities - DiGraph"] = {}
results_final["centralities - DiGraph"]["accuracy"] = {}
results_final["centralities - DiGraph"]["f1s"] = {}

results_final["centralities - MultiDiGraph"] = {}
results_final["centralities - MultiDiGraph"]["accuracy"] = {}
results_final["centralities - MultiDiGraph"]["f1s"] = {}

In [15]:
results_final

{'model': {},
 'configuration': {'folder_path': './datasets/gdlc/',
  'lr_decay': True,
  'early_stopping': False,
  'pca': True,
  'digraph_centralities': False,
  'Multi_graph_centralities': False,
  'learning_rate': 0.0001,
  'LAMBD_1': 0.0001,
  'LAMBD_2': 0.001,
  'cfg': {'multi_class': False,
   'with_network_features': False,
   'n_clients': 5,
   'n_rounds': 20,
   'config_fit': {'momentum': 0.9, 'local_epochs': 5, 'batch_size': 256}}},
 'baseline': {'accuracy': {}, 'f1s': {}},
 'centralities - PCA': {'accuracy': {}, 'f1s': {}},
 'centralities - DiGraph': {'accuracy': {}, 'f1s': {}},
 'centralities - MultiDiGraph': {'accuracy': {}, 'f1s': {}}}

In [16]:
results = {}  # a dictionary that will contain all the options and results of models
# add all options to the results dictionary, to know what options selected for obtained results
results["configuration"] = "2dt - baseline"
results["dtime"] = dtime
results["multi_class"] = cfg.multi_class
results["learning_rate"] = learning_rate
results["dataset_name"] = dataset.name
results["num_classes"] = num_classes
results["labels_names"] = labels_names
results["input_dim"] = input_dim

results["scores"] = {}
results["scores"]["server"] = {}
results["scores"]["clients"] = {}
results["scores"]["accuracy"] = {}
results["scores"]["f1s"] = {}

if not cfg.multi_class:
    results["scores"]["test_by_class"] = {}
    results["scores"]["test_by_class"]["accuracy"] = {}
    results["scores"]["test_by_class"]["f1s"] = {}
    for k in test_by_class.keys():
        results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
        results["scores"]["test_by_class"]["accuracy"][k] = {}   
        results["scores"]["test_by_class"]["f1s"][k] = {}    
        
results

{'configuration': '2dt - baseline',
 'dtime': '20240826-195245',
 'multi_class': False,
 'learning_rate': 0.0001,
 'dataset_name': 'cic_ton_iot',
 'num_classes': 2,
 'labels_names': {0: 'benign', 1: 'attack'},
 'input_dim': 38,
 'scores': {'server': {},
  'clients': {},
  'accuracy': {},
  'f1s': {},
  'test_by_class': {'accuracy': {'Benign': {},
    'injection': {},
    'xss': {},
    'password': {},
    'backdoor': {},
    'ransomware': {},
    'scanning': {},
    'ddos': {},
    'mitm': {},
    'dos': {},
    'DoS Hulk': {},
    'DoS GoldenEye': {},
    'PortScan': {},
    'DoS slowloris': {},
    'FTP-Patator': {},
    'SSH-Patator': {},
    'Bot': {},
    'DoS Slowhttptest': {},
    'bruteforce': {},
    'Infiltration': {},
    'Web Attack � Sql Injection': {},
    'Heartbleed': {}},
   'f1s': {'Benign': {},
    'injection': {},
    'xss': {},
    'password': {},
    'backdoor': {},
    'ransomware': {},
    'scanning': {},
    'ddos': {},
    'mitm': {},
    'dos': {},
    'DoS H

In [17]:

class FLClient(fl.client.NumPyClient):
    def __init__(self, cid, x_train, y_train, x_test, y_test):
        self.cid = cid
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test
        self.model = create_keras_model(input_shape=input_dim)

    def get_parameters(self, config):
        return self.model.get_weights()

    def set_parameters(self, parameters, config):
        self.model.set_weights(parameters)

    def fit(self, parameters, config):
        
        lr=float(config["lr"])
        self.model = create_keras_model(input_shape=input_dim, alpha=lr)
        self.set_parameters(parameters, config)

        
        logdir = "logs/scalars/{}/baseline/client_{}".format(dtime, self.cid)
        tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

        history = self.model.fit(self.x_train, self.y_train,
                                 epochs=config["local_epochs"],
                                 batch_size=config["batch_size"],
                                 validation_data=(self.x_test, self.y_test),
                                 verbose=0,
                                 callbacks=[tensorboard_callback])

        return self.get_parameters(config), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


    def evaluate(self, parameters, config):
        self.set_parameters(parameters, config)
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, cfg.config_fit.batch_size, verbose=0)
        return loss, len(self.x_test), {"accuracy": accuracy}


In [18]:
def generate_client_fn():
    def client_fn(cid: str):
        i = int(cid)
        return FLClient(cid, client_data[i][0], client_data[i][1], client_data[i][2], client_data[i][3]).to_client()

    return client_fn

In [19]:
def get_on_fit_config(config: DictConfig):

    def fit_config_fn(server_round: int):
        alpha = learning_rate
        if lr_decay and server_round > 5:
            alpha = alpha / (1 + 0.5 * server_round)


        return {
            "lr": alpha,
            "local_epochs": config.local_epochs,
            "batch_size": config.batch_size,
        }

    return fit_config_fn


def get_evaluate_fn(x_test_sever, y_test_server):

    def evaluate_fn(server_round: int, parameters, config):
        # eval_model = model
        eval_model = create_keras_model(input_shape=input_dim)
        eval_model.set_weights(parameters)

        
        logdir = "logs/scalars/{}/baseline/server".format(dtime) 
        # logdir = "logs/scalars/client{}_".format(config["cid"]) + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

        test_loss, test_acc = eval_model.evaluate(x_test_sever, y_test_server,
                                                  batch_size = cfg.config_fit.batch_size,
                                                  callbacks=[tensorboard_callback])
        
        
        y_pred = eval_model.predict(x_test_sever, batch_size = cfg.config_fit.batch_size)
        
        if cfg.multi_class:
            y_pred = np.argmax(y_pred, axis=1)
            scores = custom_acc_mc(y_test_server, y_pred)
        else:
            y_pred = np.transpose(y_pred)[0]
            y_pred = list(
                map(lambda x: 0 if x < 0.5 else 1, y_pred))
            scores = custom_acc_binary(y_test_server, y_pred)
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        results_final["baseline"]["accuracy"][server_round] = scores["accuracy"]
        results_final["baseline"]["f1s"][server_round] = scores["f1s"]
        
        if not cfg.multi_class:
            for k in test_by_class.keys():
                y_pred_class = eval_model.predict(test_by_class[k][0], batch_size = cfg.config_fit.batch_size, verbose = 0)
                y_pred_class = np.transpose(y_pred_class)[0]
                y_pred_class = list(map(lambda x: 0 if x < 0.5 else 1, y_pred_class))
                scores_class = custom_acc_binary(test_by_class[k][1], y_pred_class)
                results["scores"]["test_by_class"]["accuracy"][k][server_round] = scores_class["accuracy"]
                results["scores"]["test_by_class"]["f1s"][k][server_round] = scores_class["f1s"]
                
        log(INFO, f"==>> scores: {scores}")
        
        
        return test_loss, {"accuracy": test_acc, "f1s": scores["f1s"], "FPR": scores["FPR"], "FNR": scores["FNR"]}

    return evaluate_fn


In [20]:
def weighted_average(metrics):
    print(f"==>> weighted_average: {metrics}")
    results
    return metrics
    # total_examples = 0
    # federated_metrics = {k: 0 for k in metrics[0][1].keys()}
    # for num_examples, m in metrics:
    #     for k, v in m.items():
    #         federated_metrics[k] += num_examples * v
    #     total_examples += num_examples
    # return {k: v / total_examples for k, v in federated_metrics.items()}

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
    min_fit_clients=len(client_data),  # number of clients to sample for fit()
    fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
    min_evaluate_clients=0,  # number of clients to sample for evaluate()
    min_available_clients=len(client_data),  # total clients in the simulation
    # fit_metrics_aggregation_fn = weighted_average,
    # evaluate_metrics_aggregation_fn = weighted_average,
    on_fit_config_fn=get_on_fit_config(
        cfg.config_fit
    ),  # a function to execute to obtain the configuration to send to the clients during fit()
    evaluate_fn=get_evaluate_fn(test, test_labels),
)  # a function to run on the server side to evaluate the global model.

# strategy.aggregate_fit = 


## FL Simulation

In [21]:
import multiprocessing
from math import floor
history = fl.simulation.start_simulation(
    client_fn=generate_client_fn(),  # a function that spawns a particular client
    # num_clients=cfg.n_clients,  # total number of clients
    num_clients=len(client_data),  # total number of clients
    config=fl.server.ServerConfig(
        num_rounds=cfg.n_rounds
        # num_rounds=5
    ),  # minimal config for the server loop telling the number of rounds in FL
    strategy=strategy,  # our strategy of choice
    client_resources={
        # "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
        "num_cpus": 1,
        "num_gpus": 0.0,
    },
)

INFO flwr 2024-08-26 19:53:01,560 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
2024-08-26 19:53:07,086	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-08-26 19:53:09,995 | app.py:213 | Flower VCE: Ray initialized with resources: {'object_store_memory': 17426895667.0, 'node:127.0.0.1': 1.0, 'memory': 34853791335.0, 'CPU': 32.0, 'node:__internal_head__': 1.0}
INFO flwr 2024-08-26 19:53:09,997 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-08-26 19:53:09,998 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2024-08-26 19:53:10,032 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 32 actors
INFO flwr 2024-08-26 19:53:10,033 | server.py:89 | Initializing global parameters
INFO flwr 2024-08-26 19:53:10,035 | server.py:276 | Requesting initial parameters from o

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 30s 16ms/step - accuracy: 0.6784 - loss: 1.3842
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.8048 - loss: 0.6361
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

...

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - accuracy: 0.9477 - loss: 0.1661
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 24s 13ms/step - accuracy: 0.9671 - loss: 0.1344
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

In [22]:
print(f"==>> history: {history}")
print(f"==>> end of history")

==>> history: History (loss, centralized):
	round 0: 1.4706860780715942
	round 1: 0.7961853742599487
	round 2: 0.5417822003364563
	round 3: 0.5846560001373291
	round 4: 0.5962537527084351
	round 5: 0.5114448666572571
	round 6: 0.35480496287345886
	round 7: 0.3156278133392334
	round 8: 0.32065147161483765
	round 9: 0.35893362760543823
	round 10: 0.28627365827560425
	round 11: 0.3276474177837372
	round 12: 0.3916494846343994
	round 13: 0.38968425989151
	round 14: 0.26166287064552307
	round 15: 0.40797170996665955
	round 16: 0.255556583404541
	round 17: 0.2316359430551529
	round 18: 0.22039799392223358
	round 19: 0.2814200520515442
	round 20: 0.2460375875234604
History (metrics, centralized):
{'accuracy': [(0, 0.5952565670013428), (1, 0.7521862983703613), (2, 0.8328167796134949), (3, 0.790079653263092), (4, 0.7418158054351807), (5, 0.8246819972991943), (6, 0.8990072011947632), (7, 0.9142810702323914), (8, 0.9061442613601685), (9, 0.9022960066795349), (10, 0.9149001240730286), (11, 0.90867

In [23]:
# creating the directories if they don't exist
if not os.path.isdir('./results'):
    os.mkdir('./results')

# creating the directories if they don't exist
if not os.path.isdir('./results/{}'.format(dtime)):
    os.mkdir('./results/{}'.format(dtime))

# if not os.path.isdir('./results/{}'.format(dataset_name)):
#     os.mkdir('./results/{}'.format(dataset_name))

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

filename = ('./results/{}/baseline.json'.format(dtime))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results, cls=NumpyEncoder))
outfile.close()

In [24]:
filename = ('./results/{}/results_final.json'.format(dtime))
outfile = open(filename, 'w')
outfile.writelines(json.dumps(results_final, cls=NumpyEncoder))
outfile.close()

# FL - PCA

In [25]:
if pca:
    test = pd.read_parquet(folder_path + "test.parquet")

    if cfg.multi_class:
        test[dataset.label_col] = test[dataset.class_num_col]
        
    test.drop(centralities_columns[-1], axis=1, inplace=True)
    # test.drop(pca_columns, axis=1, inplace=True)
    # test.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
    # test.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
    #test.drop(["pca_1", "pca_2"], axis=1, inplace=True)

    if not cfg.multi_class:
        test_by_class = {}
        classes = test[dataset.class_col].unique()
        for class_value in classes:
            test_class = test[test[dataset.class_col] == class_value].copy()
            test_class.drop(dataset.drop_columns, axis=1, inplace=True)
            test_class.drop(dataset.weak_columns, axis=1, inplace=True)
            test_class.reset_index(drop=True, inplace=True)

            test_class_labels = test_class[dataset.label_col].to_numpy()
            test_class = test_class.drop([dataset.label_col], axis=1).to_numpy()

            test_by_class[class_value] = (test_class, test_class_labels)
        
        
    test.drop(dataset.drop_columns, axis=1, inplace=True)
    test.drop(dataset.weak_columns, axis=1, inplace=True)
    test.reset_index(drop=True, inplace=True)
        
    test_labels = test[dataset.label_col].to_numpy()
    test = test.drop([dataset.label_col], axis=1).to_numpy()
    input_dim = test.shape[1]

    client_data = []
    for client_path in clients_paths:
        client_data.append(pd.read_parquet(client_path))
        
    for i in range(len(client_data)):
        
        cdata = client_data[i]
        
        if cfg.multi_class:
            cdata[dataset.label_col] = cdata[dataset.class_num_col]
                    
        cdata.drop(centralities_columns[i], axis=1, inplace=True)
        # cdata.drop(pca_columns, axis=1, inplace=True)
        
        # cdata.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
        # cdata.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
    # cdata.drop(["pca_1", "pca_2"], axis=1, inplace=True)

        cdata.drop(dataset.drop_columns, axis=1, inplace=True)
        cdata.drop(dataset.weak_columns, axis=1, inplace=True)
        cdata.reset_index(drop=True, inplace=True)
        
        c_train, c_test = train_test_split(cdata, test_size=0.1)

        y_train = c_train[dataset.label_col].to_numpy()
        x_train = c_train.drop([dataset.label_col], axis=1).to_numpy()
        y_test = c_test[dataset.label_col].to_numpy()
        x_test = c_test.drop([dataset.label_col], axis=1).to_numpy()
        
        client_data[i] = (x_train, y_train, x_test, y_test)

In [26]:
if pca:
    results = {}  # a dictionary that will contain all the options and results of models
    # add all options to the results dictionary, to know what options selected for obtained results
    results["configuration"] = "2dt - PCA"
    results["dtime"] = time.strftime("%Y%m%d-%H%M%S")
    results["multi_class"] = cfg.multi_class
    results["learning_rate"] = learning_rate
    results["dataset_name"] = dataset.name
    results["num_classes"] = num_classes
    results["labels_names"] = labels_names
    results["input_dim"] = input_dim

    results["scores"] = {}
    results["scores"]["server"] = {}
    results["scores"]["clients"] = {}
    results["scores"]["accuracy"] = {}
    results["scores"]["f1s"] = {}

    if not cfg.multi_class:
        results["scores"]["test_by_class"] = {}
        results["scores"]["test_by_class"]["accuracy"] = {}
        results["scores"]["test_by_class"]["f1s"] = {}
        for k in test_by_class.keys():
            results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
            results["scores"]["test_by_class"]["accuracy"][k] = {}   
            results["scores"]["test_by_class"]["f1s"][k] = {}    
            
    results

In [27]:
if pca:
    model = create_keras_model(input_dim)
    model.summary()

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_44 (Conv1D)              │ (None, 38, 80)         │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_44 (MaxPooling1D) │ (None, 19, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_110         │ (None, 19, 80)         │            38 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_45 (Conv1D)              │ (None, 17, 80)         │        19,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_45 (MaxPooling1D) │ (None, 8, 80)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_111         │ (None, 8, 80)          │            16 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_22 (Flatten)            │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_88 (Dense)                │ (None, 200)            │       128,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_112         │ (None, 200)            │           400 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_89 (Dense)                │ (None, 200)            │        40,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_113         │ (None, 200)            │           400 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_90 (Dense)                │ (None, 80)             │        16,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_114         │ (None, 80)             │           160 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_91 (Dense)                │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 205,175 (801.46 KB)

 Trainable params: 205,175 (801.46 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
if pca:

    class FLClient(fl.client.NumPyClient):
        def __init__(self, cid, x_train, y_train, x_test, y_test):
            self.cid = cid
            self.x_train, self.y_train = x_train, y_train
            self.x_test, self.y_test = x_test, y_test
            self.model = create_keras_model(input_shape=input_dim)

        def get_parameters(self, config):
            return self.model.get_weights()

        def set_parameters(self, parameters, config):
            self.model.set_weights(parameters)

        def fit(self, parameters, config):
            
            lr=float(config["lr"])
            # self.model = create_keras_model(input_shape= self.x_train.shape[1], alpha=lr)
            self.model = create_keras_model(input_shape=input_dim, alpha=lr)
            # log(INFO, f"==>> config: {config}")
            # log(INFO, f"==>> float(config[lr]): {lr}")
            self.set_parameters(parameters, config)

            
            logdir = "logs/scalars/{}/PCA/client_{}".format(dtime, self.cid)
            tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

            history = self.model.fit(self.x_train, self.y_train,
                                    epochs=config["local_epochs"],
                                    batch_size=config["batch_size"],
                                    validation_data=(self.x_test, self.y_test),
                                    verbose=0,
                                    callbacks=[tensorboard_callback])

            return self.get_parameters(config), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


        def evaluate(self, parameters, config):
            self.set_parameters(parameters, config)
            loss, accuracy = self.model.evaluate(self.x_test, self.y_test, cfg.config_fit.batch_size, verbose=0)
            return loss, len(self.x_test), {"accuracy": accuracy}


In [29]:
if pca:
    def generate_client_fn():
        def client_fn(cid: str):
            i = int(cid)
            return FLClient(cid, client_data[i][0], client_data[i][1], client_data[i][2], client_data[i][3]).to_client()

        return client_fn

In [30]:
if pca:
    def get_on_fit_config(config: DictConfig):

        def fit_config_fn(server_round: int):
            alpha = learning_rate
            if lr_decay and server_round > 5:
                alpha = alpha / (1 + 0.5 * server_round)


            return {
                "lr": alpha,
                "local_epochs": config.local_epochs,
                "batch_size": config.batch_size,
            }

        return fit_config_fn


    def get_evaluate_fn(x_test_sever, y_test_server):

        def evaluate_fn(server_round: int, parameters, config):
            # eval_model = model
            eval_model = create_keras_model(input_shape=input_dim)
            eval_model.set_weights(parameters)

            
            logdir = "logs/scalars/{}/PCA/server".format(dtime) 
            # logdir = "logs/scalars/client{}_".format(config["cid"]) + datetime.now().strftime("%Y%m%d-%H%M%S")
            tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

            test_loss, test_acc = eval_model.evaluate(x_test_sever, y_test_server,
                                                    batch_size = cfg.config_fit.batch_size,
                                                    callbacks=[tensorboard_callback])
            
            
            y_pred = eval_model.predict(x_test_sever, batch_size = cfg.config_fit.batch_size)
            
            if cfg.multi_class:
                y_pred = np.argmax(y_pred, axis=1)
                scores = custom_acc_mc(y_test_server, y_pred)
            else:
                y_pred = np.transpose(y_pred)[0]
                y_pred = list(
                    map(lambda x: 0 if x < 0.5 else 1, y_pred))
                scores = custom_acc_binary(y_test_server, y_pred)
            
            
            results["scores"]["accuracy"][server_round] = test_acc
            results["scores"]["f1s"][server_round] = scores["f1s"]
            results["scores"]["server"][server_round] = scores
            
            
            results["scores"]["accuracy"][server_round] = test_acc
            results["scores"]["f1s"][server_round] = scores["f1s"]
            results["scores"]["server"][server_round] = scores
            
            results_final["centralities - PCA"]["accuracy"][server_round] = scores["accuracy"]
            results_final["centralities - PCA"]["f1s"][server_round] = scores["f1s"]
            
            if not cfg.multi_class:
                for k in test_by_class.keys():
                    y_pred_class = eval_model.predict(test_by_class[k][0], batch_size = cfg.config_fit.batch_size, verbose = 0)
                    y_pred_class = np.transpose(y_pred_class)[0]
                    y_pred_class = list(map(lambda x: 0 if x < 0.5 else 1, y_pred_class))
                    scores_class = custom_acc_binary(test_by_class[k][1], y_pred_class)
                    results["scores"]["test_by_class"]["accuracy"][k][server_round] = scores_class["accuracy"]
                    results["scores"]["test_by_class"]["f1s"][k][server_round] = scores_class["f1s"]
                    
            log(INFO, f"==>> scores: {scores}")
            
            
            return test_loss, {"accuracy": test_acc, "f1s": scores["f1s"], "FPR": scores["FPR"], "FNR": scores["FNR"]}

        return evaluate_fn


In [31]:
if pca:
    def weighted_average(metrics):
        print(f"==>> weighted_average: {metrics}")

        total_examples = 0
        federated_metrics = {k: 0 for k in metrics[0][1].keys()}
        for num_examples, m in metrics:
            for k, v in m.items():
                federated_metrics[k] += num_examples * v
            total_examples += num_examples
        return {k: v / total_examples for k, v in federated_metrics.items()}

    strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
        min_fit_clients=len(client_data),  # number of clients to sample for fit()
        fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
        min_evaluate_clients=0,  # number of clients to sample for evaluate()
        min_available_clients=len(client_data),  # total clients in the simulation
        fit_metrics_aggregation_fn = weighted_average,
        # evaluate_metrics_aggregation_fn = weighted_average,
        on_fit_config_fn=get_on_fit_config(
            cfg.config_fit
        ),  # a function to execute to obtain the configuration to send to the clients during fit()
        evaluate_fn=get_evaluate_fn(test, test_labels),
    )  # a function to run on the server side to evaluate the global model.


In [32]:
if pca:
    import multiprocessing
    from math import floor
    history = fl.simulation.start_simulation(
        client_fn=generate_client_fn(),  # a function that spawns a particular client
        # num_clients=cfg.n_clients,  # total number of clients
        num_clients=len(client_data),  # total number of clients
        config=fl.server.ServerConfig(
            num_rounds=cfg.n_rounds
            # num_rounds=5
        ),  # minimal config for the server loop telling the number of rounds in FL
        strategy=strategy,  # our strategy of choice
        client_resources={
            # "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
            "num_cpus": 1,
            "num_gpus": 0.0,
        },
    )

INFO flwr 2024-08-26 23:16:01,104 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
2024-08-26 23:16:12,109	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-08-26 23:16:14,813 | app.py:213 | Flower VCE: Ray initialized with resources: {'object_store_memory': 17216451379.0, 'node:127.0.0.1': 1.0, 'memory': 34432902759.0, 'CPU': 32.0, 'node:__internal_head__': 1.0}
INFO flwr 2024-08-26 23:16:14,814 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-08-26 23:16:14,815 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2024-08-26 23:16:14,847 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 32 actors
INFO flwr 2024-08-26 23:16:14,848 | server.py:89 | Initializing global parameters
INFO flwr 2024-08-26 23:16:14,849 | server.py:276 | Requesting initial parameters from o

1837/1837 ━━━━━━━━━━━━━━━━━━━━ 27s 14ms/step - accuracy: 0.7073 - loss: 1.3101
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step


c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

==>> weighted_average: [(489061, {'accuracy': 0.9820001721382141, 'loss': 0.1918117105960846, 'val_accuracy': 0.9824810028076172, 'val_loss': 0.17013366520404816}), (489061, {'accuracy': 0.9938187599182129, 'loss': 0.22929099202156067, 'val_accuracy': 0.9987118244171143, 'val_loss': 0.1948477327823639}), (423074, {'accuracy': 0.9691803455352783, 'loss': 0.2993837296962738, 'val_accuracy': 0.9683677554130554, 'val_loss': 0.2767115533351898}), (489061, {'accuracy': 0.9298921823501587, 'loss': 0.3651159107685089, 'val_accuracy': 0.931617021560669, 'val_loss': 0.33890500664711}), (454119, {'accuracy': 0.9994869232177734, 'loss': 0.037658803164958954, 'val_accuracy': 0.9994649291038513, 'val_loss': 0.03111143968999386}), (489061, {'accuracy': 0.9777287244796753, 'loss': 0.217156320810318, 'val_accuracy': 0.9813952445983887, 'val_loss': 0.18928739428520203}), (454118, {'accuracy': 0.99885493516922, 'loss': 0.0952146053314209, 'val_accuracy': 0.9989694356918335, 'val_loss': 0.0802978724241256

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

...

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

==>> weighted_average: [(454118, {'accuracy': 0.9998172521591187, 'loss': 0.003989102318882942, 'val_accuracy': 0.9998810887336731, 'val_loss': 0.004034675192087889}), (489061, {'accuracy': 0.9999958872795105, 'loss': 0.0030292747542262077, 'val_accuracy': 0.9999815821647644, 'val_loss': 0.0030029399786144495}), (489061, {'accuracy': 0.9999897480010986, 'loss': 0.003178059123456478, 'val_accuracy': 1.0, 'val_loss': 0.003099992172792554}), (489061, {'accuracy': 0.9999529719352722, 'loss': 0.0032958420924842358, 'val_accuracy': 0.9999263882637024, 'val_loss': 0.0033106564078480005}), (489061, {'accuracy': 0.9999734163284302, 'loss': 0.003236962715163827, 'val_accuracy': 0.9999815821647644, 'val_loss': 0.0034373842645436525}), (454119, {'accuracy': 0.9999405145645142, 'loss': 0.0034198639914393425, 'val_accuracy': 0.9999603629112244, 'val_loss': 0.003301547607406974}), (423074, {'accuracy': 0.9989292621612549, 'loss': 0.006949540693312883, 'val_accuracy': 0.9991278052330017, 'val_loss': 0

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

==>> weighted_average: [(423074, {'accuracy': 0.9992979764938354, 'loss': 0.005504346452653408, 'val_accuracy': 0.9992980360984802, 'val_loss': 0.005323163699358702}), (489061, {'accuracy': 0.9999938607215881, 'loss': 0.003006475046277046, 'val_accuracy': 1.0, 'val_loss': 0.0029672097880393267}), (489061, {'accuracy': 0.9999877214431763, 'loss': 0.0029974619392305613, 'val_accuracy': 0.999944806098938, 'val_loss': 0.0031361952424049377}), (489061, {'accuracy': 0.999979555606842, 'loss': 0.0031504782382398844, 'val_accuracy': 1.0, 'val_loss': 0.0030510600190609694}), (489061, {'accuracy': 0.999965250492096, 'loss': 0.0031520212069153786, 'val_accuracy': 0.99957674741745, 'val_loss': 0.00437974976375699}), (454119, {'accuracy': 0.9999008774757385, 'loss': 0.003478687023743987, 'val_accuracy': 0.9999405741691589, 'val_loss': 0.00344035099260509}), (454118, {'accuracy': 0.9998326301574707, 'loss': 0.0038876477628946304, 'val_accuracy': 0.9998414516448975, 'val_loss': 0.0037856739945709705}

c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Administrateur\Desktop\FL-GDLC\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

In [33]:
if pca:
    print(f"==>> history: {history}")
    print(f"==>> end of history")

==>> history: History (loss, centralized):
	round 0: 1.332646369934082
	round 1: 1.1088956594467163
	round 2: 1.081861138343811
	round 3: 1.0835907459259033
	round 4: 1.104138731956482
	round 5: 1.08480966091156
	round 6: 0.47782090306282043
	round 7: 0.37566450238227844
	round 8: 0.45157691836357117
	round 9: 0.4127289354801178
	round 10: 0.3087701201438904
	round 11: 0.32587432861328125
	round 12: 0.299843966960907
	round 13: 0.30836156010627747
	round 14: 0.3203623294830322
	round 15: 0.3913596570491791
	round 16: 0.18831127882003784
	round 17: 0.12002314627170563
	round 18: 0.16062572598457336
	round 19: 0.12142322957515717
	round 20: 0.14441846311092377
History (metrics, distributed, fit):
{'accuracy': [(1, 0.9828006200638154), (2, 0.9875607895778749), (3, 0.9937653869509501), (4, 0.9948122576657221), (5, 0.9986165583081782), (6, 0.9994239733593356), (7, 0.999645452065763), (8, 0.9995925155248128), (9, 0.999735033686714), (10, 0.9997577312015618), (11, 0.999764114907052), (12, 0.9

In [34]:
if pca:
    filename = ('./results/{}/pca.json'.format(dtime))
    outfile = open(filename, 'w')
    outfile.writelines(json.dumps(results, cls=NumpyEncoder))
    outfile.close()

In [35]:
if pca:
    filename = ('./results/{}/results_final.json'.format(dtime))
    outfile = open(filename, 'w')
    outfile.writelines(json.dumps(results_final, cls=NumpyEncoder))
    outfile.close()

# Centralities - DiGraph

In [36]:
if digraph_centralities:
    test = pd.read_parquet(folder_path + "test.parquet")

    if cfg.multi_class:
        test[dataset.label_col] = test[dataset.class_num_col]
        
    # test.drop(centralities_columns[-1], axis=1, inplace=True)
    test.drop(pca_columns, axis=1, inplace=True)
    # test.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
    # test.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
    # test.drop(["pca_1", "pca_2"], axis=1, inplace=True)

    if not cfg.multi_class:
        test_by_class = {}
        classes = test[dataset.class_col].unique()
        for class_value in classes:
            test_class = test[test[dataset.class_col] == class_value].copy()
            test_class.drop(dataset.drop_columns, axis=1, inplace=True)
            test_class.drop(dataset.weak_columns, axis=1, inplace=True)
            test_class.reset_index(drop=True, inplace=True)

            test_class_labels = test_class[dataset.label_col].to_numpy()
            test_class = test_class.drop([dataset.label_col], axis=1).to_numpy()

            test_by_class[class_value] = (test_class, test_class_labels)
        
        
    test.drop(dataset.drop_columns, axis=1, inplace=True)
    test.drop(dataset.weak_columns, axis=1, inplace=True)
    test.reset_index(drop=True, inplace=True)
        
    test_labels = test[dataset.label_col].to_numpy()
    test = test.drop([dataset.label_col], axis=1).to_numpy()
    input_dim = test.shape[1]

    client_data = []
    for client_path in clients_paths:
        client_data.append(pd.read_parquet(client_path))
        
    for i in range(len(client_data)):
        
        cdata = client_data[i]
        
        if cfg.multi_class:
            cdata[dataset.label_col] = cdata[dataset.class_num_col]
                    
        # cdata.drop(centralities_columns[i], axis=1, inplace=True)
        cdata.drop(pca_columns, axis=1, inplace=True)
        # cdata.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
        # cdata.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)
        # cdata.drop(["pca_1", "pca_2"], axis=1, inplace=True)

        cdata.drop(dataset.drop_columns, axis=1, inplace=True)
        cdata.drop(dataset.weak_columns, axis=1, inplace=True)
        cdata.reset_index(drop=True, inplace=True)
        
        c_train, c_test = train_test_split(cdata, test_size=0.1)

        y_train = c_train[dataset.label_col].to_numpy()
        x_train = c_train.drop([dataset.label_col], axis=1).to_numpy()
        y_test = c_test[dataset.label_col].to_numpy()
        x_test = c_test.drop([dataset.label_col], axis=1).to_numpy()
        
        client_data[i] = (x_train, y_train, x_test, y_test)

In [37]:
if digraph_centralities:
    results = {}  # a dictionary that will contain all the options and results of models
    # add all options to the results dictionary, to know what options selected for obtained results
    results["configuration"] = "2dt - Centralities - DiGraph"
    results["dtime"] = time.strftime("%Y%m%d-%H%M%S")
    results["multi_class"] = cfg.multi_class
    results["learning_rate"] = learning_rate
    results["dataset_name"] = dataset.name
    results["num_classes"] = num_classes
    results["labels_names"] = labels_names
    results["input_dim"] = input_dim

    results["scores"] = {}
    results["scores"]["server"] = {}
    results["scores"]["clients"] = {}
    results["scores"]["accuracy"] = {}
    results["scores"]["f1s"] = {}

    if not cfg.multi_class:
        results["scores"]["test_by_class"] = {}
        results["scores"]["test_by_class"]["accuracy"] = {}
        results["scores"]["test_by_class"]["f1s"] = {}
        for k in test_by_class.keys():
            results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
            results["scores"]["test_by_class"]["accuracy"][k] = {}   
            results["scores"]["test_by_class"]["f1s"][k] = {}    
            
    results

In [38]:
if digraph_centralities:
    model = create_keras_model(input_dim)
    model.summary()

In [39]:
if digraph_centralities:

    class FLClient(fl.client.NumPyClient):
        def __init__(self, cid, x_train, y_train, x_test, y_test):
            self.cid = cid
            self.x_train, self.y_train = x_train, y_train
            self.x_test, self.y_test = x_test, y_test
            self.model = create_keras_model(input_shape=input_dim)

        def get_parameters(self, config):
            return self.model.get_weights()

        def set_parameters(self, parameters, config):
            self.model.set_weights(parameters)

        def fit(self, parameters, config):
            
            lr=float(config["lr"])
            # self.model = create_keras_model(input_shape= self.x_train.shape[1], alpha=lr)
            self.model = create_keras_model(input_shape=input_dim, alpha=lr)
            # log(INFO, f"==>> config: {config}")
            # log(INFO, f"==>> float(config[lr]): {lr}")
            self.set_parameters(parameters, config)

            
            logdir = "logs/scalars/{}/digraph/client_{}".format(dtime, self.cid)
            tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

            history = self.model.fit(self.x_train, self.y_train,
                                    epochs=config["local_epochs"],
                                    batch_size=config["batch_size"],
                                    validation_data=(self.x_test, self.y_test),
                                    verbose=0,
                                    callbacks=[tensorboard_callback])

            return self.get_parameters(config), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


        def evaluate(self, parameters, config):
            self.set_parameters(parameters, config)
            loss, accuracy = self.model.evaluate(self.x_test, self.y_test, cfg.config_fit.batch_size, verbose=0)
            return loss, len(self.x_test), {"accuracy": accuracy}


In [40]:
if digraph_centralities:
    def generate_client_fn():
        def client_fn(cid: str):
            i = int(cid)
            return FLClient(cid, client_data[i][0], client_data[i][1], client_data[i][2], client_data[i][3]).to_client()

        return client_fn

In [41]:
if digraph_centralities:
    def get_on_fit_config(config: DictConfig):

        def fit_config_fn(server_round: int):
            alpha = learning_rate
            # if server_round > 5:
            #     alpha = alpha / (1 + 0.5 * server_round)


            return {
                "lr": alpha,
                "local_epochs": config.local_epochs,
                "batch_size": config.batch_size,
            }

        return fit_config_fn


    def get_evaluate_fn(x_test_sever, y_test_server):

        def evaluate_fn(server_round: int, parameters, config):
            # eval_model = model
            eval_model = create_keras_model(input_shape=input_dim)
            eval_model.set_weights(parameters)

            
            logdir = "logs/scalars/{}/digraph/server".format(dtime) 
            # logdir = "logs/scalars/client{}_".format(config["cid"]) + datetime.now().strftime("%Y%m%d-%H%M%S")
            tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

            test_loss, test_acc = eval_model.evaluate(x_test_sever, y_test_server,
                                                    batch_size = cfg.config_fit.batch_size,
                                                    callbacks=[tensorboard_callback])
            
            
            y_pred = eval_model.predict(x_test_sever, batch_size = cfg.config_fit.batch_size)
            
            if cfg.multi_class:
                y_pred = np.argmax(y_pred, axis=1)
                scores = custom_acc_mc(y_test_server, y_pred)
            else:
                y_pred = np.transpose(y_pred)[0]
                y_pred = list(
                    map(lambda x: 0 if x < 0.5 else 1, y_pred))
                scores = custom_acc_binary(y_test_server, y_pred)
            
            
            results["scores"]["accuracy"][server_round] = test_acc
            results["scores"]["f1s"][server_round] = scores["f1s"]
            results["scores"]["server"][server_round] = scores
            
            
            results["scores"]["accuracy"][server_round] = test_acc
            results["scores"]["f1s"][server_round] = scores["f1s"]
            results["scores"]["server"][server_round] = scores
            
            results_final["centralities - DiGraph"]["accuracy"][server_round] = scores["accuracy"]
            results_final["centralities - DiGraph"]["f1s"][server_round] = scores["f1s"]
            
            if not cfg.multi_class:
                for k in test_by_class.keys():
                    y_pred_class = eval_model.predict(test_by_class[k][0], batch_size = cfg.config_fit.batch_size, verbose = 0)
                    y_pred_class = np.transpose(y_pred_class)[0]
                    y_pred_class = list(map(lambda x: 0 if x < 0.5 else 1, y_pred_class))
                    scores_class = custom_acc_binary(test_by_class[k][1], y_pred_class)
                    results["scores"]["test_by_class"]["accuracy"][k][server_round] = scores_class["accuracy"]
                    results["scores"]["test_by_class"]["f1s"][k][server_round] = scores_class["f1s"]
                    
            log(INFO, f"==>> scores: {scores}")
            
            
            return test_loss, {"accuracy": test_acc, "f1s": scores["f1s"], "FPR": scores["FPR"], "FNR": scores["FNR"]}

        return evaluate_fn


In [42]:
if digraph_centralities:
    def weighted_average(metrics):
        print(f"==>> weighted_average: {metrics}")

        total_examples = 0
        federated_metrics = {k: 0 for k in metrics[0][1].keys()}
        for num_examples, m in metrics:
            for k, v in m.items():
                federated_metrics[k] += num_examples * v
            total_examples += num_examples
        return {k: v / total_examples for k, v in federated_metrics.items()}

    strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
        min_fit_clients=len(client_data),  # number of clients to sample for fit()
        fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
        min_evaluate_clients=0,  # number of clients to sample for evaluate()
        min_available_clients=len(client_data),  # total clients in the simulation
        fit_metrics_aggregation_fn = weighted_average,
        # evaluate_metrics_aggregation_fn = weighted_average,
        on_fit_config_fn=get_on_fit_config(
            cfg.config_fit
        ),  # a function to execute to obtain the configuration to send to the clients during fit()
        evaluate_fn=get_evaluate_fn(test, test_labels),
    )  # a function to run on the server side to evaluate the global model.


In [43]:
if digraph_centralities:
    import multiprocessing
    from math import floor
    history = fl.simulation.start_simulation(
        client_fn=generate_client_fn(),  # a function that spawns a particular client
        # num_clients=cfg.n_clients,  # total number of clients
        num_clients=len(client_data),  # total number of clients
        config=fl.server.ServerConfig(
            num_rounds=cfg.n_rounds
            # num_rounds=5
        ),  # minimal config for the server loop telling the number of rounds in FL
        strategy=strategy,  # our strategy of choice
        client_resources={
            # "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
            "num_cpus": 1,
            "num_gpus": 0.0,
        },
    )

In [44]:
if digraph_centralities:
    print(f"==>> history: {history}")
    print(f"==>> end of history")

In [45]:
if digraph_centralities:
    filename = ('./results/{}/digraph.json'.format(dtime))
    outfile = open(filename, 'w')
    outfile.writelines(json.dumps(results, cls=NumpyEncoder))
    outfile.close()

In [46]:
if digraph_centralities:
    filename = ('./results/{}/results_final.json'.format(dtime))
    outfile = open(filename, 'w')
    outfile.writelines(json.dumps(results_final, cls=NumpyEncoder))
    outfile.close()

# Centralities - MultiDiGraph

In [47]:
# test = pd.read_parquet(folder_path + "test.parquet")

# if cfg.multi_class:
#     test[dataset.label_col] = test[dataset.class_num_col]
    
# test.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
# # test.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)

# if not cfg.multi_class:
#     test_by_class = {}
#     classes = test[dataset.class_col].unique()
#     for class_value in classes:
#         test_class = test[test[dataset.class_col] == class_value].copy()
#         test_class.drop(dataset.drop_columns, axis=1, inplace=True)
#         test_class.drop(dataset.weak_columns, axis=1, inplace=True)
#         test_class.reset_index(drop=True, inplace=True)

#         test_class_labels = test_class[dataset.label_col].to_numpy()
#         test_class = test_class.drop([dataset.label_col], axis=1).to_numpy()

#         test_by_class[class_value] = (test_class, test_class_labels)
    
    
# test.drop(dataset.drop_columns, axis=1, inplace=True)
# test.drop(dataset.weak_columns, axis=1, inplace=True)
# test.reset_index(drop=True, inplace=True)
    
# test_labels = test[dataset.label_col].to_numpy()
# test = test.drop([dataset.label_col], axis=1).to_numpy()
# input_dim = test.shape[1]

# client_data = []
# for client_path in clients_paths:
#     client_data.append(pd.read_parquet(client_path))
    
# for i in range(len(client_data)):
    
#     cdata = client_data[i]
    
#     if cfg.multi_class:
#         cdata[dataset.label_col] = cdata[dataset.class_num_col]
        
#     cdata.drop(["src_degree", "dst_degree", "src_betweenness", "dst_betweenness", "src_pagerank", "dst_pagerank"], axis=1, inplace=True)
#     # cdata.drop(["src_multidigraph_degree", "dst_multidigraph_degree", "src_multidigraph_betweenness", "dst_multidigraph_betweenness", "src_multidigraph_pagerank", "dst_multidigraph_pagerank"], axis=1, inplace=True)

#     cdata.drop(dataset.drop_columns, axis=1, inplace=True)
#     cdata.drop(dataset.weak_columns, axis=1, inplace=True)
#     cdata.reset_index(drop=True, inplace=True)
    
#     c_train, c_test = train_test_split(cdata, test_size=0.1)

#     y_train = c_train[dataset.label_col].to_numpy()
#     x_train = c_train.drop([dataset.label_col], axis=1).to_numpy()
#     y_test = c_test[dataset.label_col].to_numpy()
#     x_test = c_test.drop([dataset.label_col], axis=1).to_numpy()
    
#     client_data[i] = (x_train, y_train, x_test, y_test)

In [48]:
# test.head()

In [49]:
# results = {}  # a dictionary that will contain all the options and results of models
# # add all options to the results dictionary, to know what options selected for obtained results
# results["configuration"] = "2dt - Centralities - MultiDiGraph"
# results["dtime"] = time.strftime("%Y%m%d-%H%M%S")
# results["multi_class"] = cfg.multi_class
# results["learning_rate"] = learning_rate
# results["dataset_name"] = dataset.name
# results["num_classes"] = num_classes
# results["labels_names"] = labels_names
# results["input_dim"] = input_dim

# results["scores"] = {}
# results["scores"]["server"] = {}
# results["scores"]["clients"] = {}
# results["scores"]["accuracy"] = {}
# results["scores"]["f1s"] = {}

# if not cfg.multi_class:
#     results["scores"]["test_by_class"] = {}
#     results["scores"]["test_by_class"]["accuracy"] = {}
#     results["scores"]["test_by_class"]["f1s"] = {}
#     for k in test_by_class.keys():
#         results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
#         results["scores"]["test_by_class"]["accuracy"][k] = {}   
#         results["scores"]["test_by_class"]["f1s"][k] = {}    
        
# results

In [50]:
# model = create_keras_model(input_dim)
# model.summary()

In [51]:

# class FLClient(fl.client.NumPyClient):
#     def __init__(self, cid, x_train, y_train, x_test, y_test):
#         self.cid = cid
#         self.x_train, self.y_train = x_train, y_train
#         self.x_test, self.y_test = x_test, y_test
#         self.model = create_keras_model(input_shape=input_dim)

#     def get_parameters(self, config):
#         return self.model.get_weights()

#     def set_parameters(self, parameters, config):
#         self.model.set_weights(parameters)

#     def fit(self, parameters, config):
        
#         lr=float(config["lr"])
#         # self.model = create_keras_model(input_shape= self.x_train.shape[1], alpha=lr)
#         self.model = create_keras_model(input_shape=input_dim, alpha=lr)
#         # log(INFO, f"==>> config: {config}")
#         # log(INFO, f"==>> float(config[lr]): {lr}")
#         self.set_parameters(parameters, config)

        
#         logdir = "logs/scalars/{}/multidigraph/client_{}".format(dtime, self.cid)
#         tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

#         history = self.model.fit(self.x_train, self.y_train,
#                                  epochs=config["local_epochs"],
#                                  batch_size=config["batch_size"],
#                                  validation_data=(self.x_test, self.y_test),
#                                  verbose=0,
#                                  callbacks=[tensorboard_callback])

#         return self.get_parameters(config), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


#     def evaluate(self, parameters, config):
#         self.set_parameters(parameters, config)
#         loss, accuracy = self.model.evaluate(self.x_test, self.y_test, cfg.config_fit.batch_size, verbose=0)
#         return loss, len(self.x_test), {"accuracy": accuracy}


In [52]:
# def generate_client_fn():
#     def client_fn(cid: str):
#         i = int(cid)
#         return FLClient(cid, client_data[i][0], client_data[i][1], client_data[i][2], client_data[i][3]).to_client()

#     return client_fn

In [53]:
# def get_on_fit_config(config: DictConfig):

#     def fit_config_fn(server_round: int):
#         alpha = learning_rate
        # if lr_decay and server_round > 5:
        #     alpha = alpha / (1 + 0.5 * server_round)
#         if server_round > 5:
#             alpha = alpha / (1 + 0.5 * server_round)


#         return {
#             "lr": alpha,
#             "local_epochs": config.local_epochs,
#             "batch_size": config.batch_size,
#         }

#     return fit_config_fn


# def get_evaluate_fn(x_test_sever, y_test_server):

#     def evaluate_fn(server_round: int, parameters, config):
#         # eval_model = model
#         eval_model = create_keras_model(input_shape=input_dim)
#         eval_model.set_weights(parameters)

        
#         logdir = "logs/scalars/{}/multidigraph/server".format(dtime) 
#         # logdir = "logs/scalars/client{}_".format(config["cid"]) + datetime.now().strftime("%Y%m%d-%H%M%S")
#         tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)

#         test_loss, test_acc = eval_model.evaluate(x_test_sever, y_test_server,
#                                                   batch_size = cfg.config_fit.batch_size,
#                                                   callbacks=[tensorboard_callback])
        
        
#         y_pred = eval_model.predict(x_test_sever, batch_size = cfg.config_fit.batch_size)
        
#         if cfg.multi_class:
#             y_pred = np.argmax(y_pred, axis=1)
#             scores = custom_acc_mc(y_test_server, y_pred)
#         else:
#             y_pred = np.transpose(y_pred)[0]
#             y_pred = list(
#                 map(lambda x: 0 if x < 0.5 else 1, y_pred))
#             scores = custom_acc_binary(y_test_server, y_pred)
        
        
#         results["scores"]["accuracy"][server_round] = test_acc
#         results["scores"]["f1s"][server_round] = scores["f1s"]
#         results["scores"]["server"][server_round] = scores
        
        
#         results["scores"]["accuracy"][server_round] = test_acc
#         results["scores"]["f1s"][server_round] = scores["f1s"]
#         results["scores"]["server"][server_round] = scores
        
#         results_final["centralities - MultiDiGraph"]["accuracy"][server_round] = scores["accuracy"]
#         results_final["centralities - MultiDiGraph"]["f1s"][server_round] = scores["f1s"]
        
#         if not cfg.multi_class:
#             for k in test_by_class.keys():
#                 y_pred_class = eval_model.predict(test_by_class[k][0], batch_size = cfg.config_fit.batch_size, verbose = 0)
#                 y_pred_class = np.transpose(y_pred_class)[0]
#                 y_pred_class = list(map(lambda x: 0 if x < 0.5 else 1, y_pred_class))
#                 scores_class = custom_acc_binary(test_by_class[k][1], y_pred_class)
#                 results["scores"]["test_by_class"]["accuracy"][k][server_round] = scores_class["accuracy"]
#                 results["scores"]["test_by_class"]["f1s"][k][server_round] = scores_class["f1s"]
                
#         log(INFO, f"==>> scores: {scores}")
        
        
#         return test_loss, {"accuracy": test_acc, "f1s": scores["f1s"], "FPR": scores["FPR"], "FNR": scores["FNR"]}

#     return evaluate_fn


In [54]:
# def weighted_average(metrics):
#     print(f"==>> weighted_average: {metrics}")

#     total_examples = 0
#     federated_metrics = {k: 0 for k in metrics[0][1].keys()}
#     for num_examples, m in metrics:
#         for k, v in m.items():
#             federated_metrics[k] += num_examples * v
#         total_examples += num_examples
#     return {k: v / total_examples for k, v in federated_metrics.items()}

# strategy = fl.server.strategy.FedAvg(
#     fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
#     min_fit_clients=len(client_data),  # number of clients to sample for fit()
#     fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
#     min_evaluate_clients=0,  # number of clients to sample for evaluate()
#     min_available_clients=len(client_data),  # total clients in the simulation
#     fit_metrics_aggregation_fn = weighted_average,
#     # evaluate_metrics_aggregation_fn = weighted_average,
#     on_fit_config_fn=get_on_fit_config(
#         cfg.config_fit
#     ),  # a function to execute to obtain the configuration to send to the clients during fit()
#     evaluate_fn=get_evaluate_fn(test, test_labels),
# )  # a function to run on the server side to evaluate the global model.


In [55]:
# import multiprocessing
# from math import floor
# history = fl.simulation.start_simulation(
#     client_fn=generate_client_fn(),  # a function that spawns a particular client
#     # num_clients=cfg.n_clients,  # total number of clients
#     num_clients=len(client_data),  # total number of clients
#     config=fl.server.ServerConfig(
#         num_rounds=cfg.n_rounds
#         # num_rounds=5
#     ),  # minimal config for the server loop telling the number of rounds in FL
#     strategy=strategy,  # our strategy of choice
#     client_resources={
#         "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
#         "num_gpus": 0.0,
#     },
# )

In [56]:
# print(f"==>> history: {history}")
# print(f"==>> end of history")

In [57]:
# filename = ('./results/{}/multidigraph.json'.format(dtime))
# outfile = open(filename, 'w')
# outfile.writelines(json.dumps(results, cls=NumpyEncoder))
# outfile.close()

In [58]:
# filename = ('./results/{}/results_final.json'.format(dtime))
# outfile = open(filename, 'w')
# outfile.writelines(json.dumps(results_final, cls=NumpyEncoder))
# outfile.close()